<a href="https://colab.research.google.com/github/federico2879/MLDL2024_semantic_segmentation/blob/master/training/training-bisenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 552 (delta 117), reused 150 (delta 93), pack-reused 349
Receiving objects: 100% (552/552), 272.11 KiB | 2.89 MiB/s, done.
Resolving deltas: 100% (324/324), done.


In [2]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.datasets.gta5 import GTA5
from MLDL2024_semantic_segmentation.models.bisenet.build_bisenet import *
from MLDL2024_semantic_segmentation.train import * 
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.IOU import * 

## Setup

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Setup fixed parameters
num_epochs = 50
num_classes = 19
batch_size = 4

## Dataset

In [5]:
# Transformations
transform_image = {
    'train': transforms.Compose([
        transforms.Resize((1280, 720)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((1024, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}
                   
transform_target = {
    'train': transforms.Compose([
        transforms.Resize((1280, 720), interpolation=InterpolationMode.NEAREST),
    ]),
    'test': transforms.Compose([
        transforms.Resize((1024, 512), interpolation=InterpolationMode.NEAREST)
    ])
}

In [6]:
# Create dataloader
dataset_train = GTA5('/kaggle/input/mldl-gta5/GTA', 
                     transform = transform_image['train'], 
                     label_transform = transform_target['train'])
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', 
                          split = 'val', transform = transform_image['test'], 
                          label_transform = transform_target['test'])
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

## Network, Loss, Optimizer

In [7]:
# Inizialization of the model
model = BiSeNet(num_classes=num_classes, context_path="resnet18").to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 143MB/s] 
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 155MB/s]  


In [8]:
# Define loss and optimizer
loss_fn =  nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=2.5e-2, momentum=0.9, 
                            weight_decay=1e-4)

## Training

In [9]:
# metrics

meanIOU_tr = np.zeros((num_epochs,1))
IOU_tr = np.zeros((num_epochs,num_classes))
loss_tr = np.zeros((num_epochs,1))


meanIOU_val = np.zeros((num_epochs,1))
IOU_val = np.zeros((num_epochs,num_classes))
loss_val = np.zeros((num_epochs,1))

In [10]:
## Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
for epoch in range(num_epochs):
    poly_lr_scheduler(optimizer, 2.5e-2, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    meanIOU_tr[epoch], IOU_tr[epoch,:], loss_tr[epoch] = train(model, optimizer, dataloader_train, loss_fn, num_classes, 0)
    meanIOU_val[epoch], IOU_val[epoch,:], loss_val[epoch] = test(model, dataloader_val, loss_fn, num_classes, 0)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU_val[epoch,0]:.2f}")

    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'meanIOU_tr': meanIOU_tr,
        'IOU_tr': IOU_tr,
        'loss_tr': loss_tr,
        'meanIOU_val': meanIOU_val,
        'IOU_val': IOU_val,
        'loss_val': loss_val
        },"checkpoint.pth.tar")

epoch: 1, Validation IOU: 0.10
epoch: 2, Validation IOU: 0.16
epoch: 3, Validation IOU: 0.16
epoch: 4, Validation IOU: 0.17
epoch: 5, Validation IOU: 0.15
epoch: 6, Validation IOU: 0.17
epoch: 7, Validation IOU: 0.17
epoch: 8, Validation IOU: 0.17
epoch: 9, Validation IOU: 0.17
epoch: 10, Validation IOU: 0.16
epoch: 11, Validation IOU: 0.19
epoch: 12, Validation IOU: 0.18
epoch: 13, Validation IOU: 0.18
epoch: 14, Validation IOU: 0.19
epoch: 15, Validation IOU: 0.18
epoch: 16, Validation IOU: 0.19
epoch: 17, Validation IOU: 0.19
epoch: 18, Validation IOU: 0.20
epoch: 19, Validation IOU: 0.20
epoch: 20, Validation IOU: 0.19
epoch: 21, Validation IOU: 0.17
epoch: 22, Validation IOU: 0.21
epoch: 23, Validation IOU: 0.18
epoch: 24, Validation IOU: 0.19
epoch: 25, Validation IOU: 0.18
epoch: 26, Validation IOU: 0.20
epoch: 27, Validation IOU: 0.18
epoch: 28, Validation IOU: 0.20
epoch: 29, Validation IOU: 0.20
epoch: 30, Validation IOU: 0.20
epoch: 31, Validation IOU: 0.20
epoch: 32, Valida

In [12]:
# final print

print(f"Final mIOU: {meanIOU_val[epoch,0]:.2f}")
print("Final IOU classes")
print(IOU_val[epoch,:])

Final mIOU: 0.20
Final IOU classes
[6.27257253e-01 7.78697130e-02 6.78631755e-01 3.73882735e-02
 5.13530056e-02 7.57407840e-02 4.29987291e-02 2.59653349e-02
 6.83283929e-01 6.06674877e-02 6.62137662e-01 1.56976907e-01
 4.00673131e-05 4.06169732e-01 7.42920007e-02 5.30663884e-02
 9.66251819e-03 3.43539955e-03 2.45665086e-05]


In [13]:
# writing csv
import csv

with open('meanIOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_tr)

with open('IOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_tr)

with open('loss_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_tr)

with open('meanIOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_val)

with open('IOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_val)

with open('loss_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_val)